In [66]:
import pdfplumber
import re
from langchain_groq import ChatGroq

In [67]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

In [68]:
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    all_text = "\n".join(pages) if pages else ""
    # print(all_text)
    return all_text

In [69]:
pdf_path = "C:/Users/Admin/Downloads/Mandar_Bhalerao_IISc.pdf"
pdf_data = extract_text_from_pdf(pdf_path)
# output_path = process_resume(pdf_path)
# print(f"Cold email prompt saved at: {output_path}")

In [70]:
from langchain_core.prompts import PromptTemplate
# (NO PREAMBLE) means dont give that initial text like Here is your response.
prompt_extract = PromptTemplate.from_template(
        """
        ### PDF DATA OBTAINED FROM RESUME:
        {pdf_data}
        ### INSTRUCTION:
        The data is from the resume of a person.
        Your job is to extract all the details of this person and return them in JSON format containing the 
        following keys: `name`, `education`, `experience`, `projects`,`skills`, and `achievements`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

 # Your job is to extract the job postings and return them in JSON format containing the 
 #        following keys: `role`, `experience`, `skills` and `description`.


 #        "name": name,
 #        "education": education,
 #        "experience": experience,
 #        "projects": projects,
 #        "skills": skills,
 #        "achievements": achievements


# def generate_cold_email(details):
#     return f"""
# You are {details['name']}, a graduate from {details['education']}. Your professional experience includes {details['experience']}. You have led projects such as {details['projects']} and are skilled in {details['skills']}. You have also achieved {details['achievements']}.

# Your task is to write a cold email to a potential employer or client, showcasing your skills and experiences detailed above. Mention your hands-on experience with technologies and how you can contribute to solving real-world problems.

# Remember, you are {details['name']}, ready to make a significant impact in your new role.
#     """

In [74]:
chain_extract = prompt_extract | llm    # this will form a langchain chain ie you are getting a prompt and passing it to LLM 
res = chain_extract.invoke(input={'pdf_data':pdf_data})
# print(res.content)

# we got the json format of the job description

In [75]:
type(res.content)

str

In [76]:
len(res.content)

4805

In [77]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
# json_res

In [78]:
type(json_res)

dict

In [79]:
# # so whenever there is a job posting, we will extract this skills from the job 
# # and we will match it with one or multiple of these technologies mentioned in the csv file and it will retrive those portfolio urls
# # which we will use while writing an email

# import pandas as pd

# df = pd.read_csv("my_portfolio.csv")
# # df


# import uuid
# import chromadb

# # when you use Client, it will create a chromadb in memory
# # but when we use PersistentClient it will create a chromadb on a disk ie it will be stored in our current folder so that we can retrive it anytime
# client = chromadb.PersistentClient('vectorstore')
# collection = client.get_or_create_collection(name="portfolio")


# if not collection.count():     # this means if collection does not have any count ie if it is being created for the first time
#     for _, row in df.iterrows():    # then you iterate through all your dataframe rows, and for each row, you are adding a document
#         collection.add(documents=row["Techstack"],
#                        metadatas={"links": row["Links"]},
#                        ids=[str(uuid.uuid4())])

In [80]:
job = json_res

In [81]:
job

{'name': 'Mandar Bhalerao',
 'education': [{'institution': 'Indian Institute of Science',
   'degree': 'Master of Technology - Computer Science and Automation',
   'cgpa': '7.30/10.0',
   'duration': 'Aug 2023 – Jul 2025',
   'location': 'Bangalore, Karnataka'},
  {'institution': 'P.E.S. Modern College of Engineering',
   'degree': 'Bachelor of Engineering - Computer Engineering',
   'cgpa': '9.34/10.0',
   'duration': 'Aug 2019 – Jul 2023',
   'location': 'Pune, Maharashtra'}],
 'experience': [{'company': 'NeuroPixel.AI',
   'position': 'Deep Learning Research Intern',
   'duration': 'May 2024 – July 2024',
   'achievements': ['Worked on optimization of Stable Diffusion models to improve performance, achieving significant efficiency gains.',
    'Implemented the Hyper-SD framework to enhance image synthesis efficiency by Knowledge Distillation techniques.',
    'Combined the advantages of Trajectory Preserving and Reformulation Distillation techniques for faster inference.',
    'Trai

In [82]:
# def generate_cold_email(details):
#     # Extract name
#     name = details.get('name', 'Candidate')

#     # Extract education details
#     education_list = details.get('education', [])
#     if education_list:
#         education_details = ', '.join([f"{edu.get('degree', 'Unknown degree')} from {edu.get('institution', 'Unknown institution')} ({edu.get('duration', 'Unknown duration')})" for edu in education_list])
#     else:
#         education_details = 'No education details provided'

#     # Extract skills details
#     skills_list = details.get('skills', [])
#     if skills_list:
#         skills_details = ', '.join([', '.join(skill.get('tools', [])) for skill in skills_list])
#     else:
#         skills_details = 'No skills listed'

#     # Extract experience details
#     experience_list = details.get('experience', [])
#     if experience_list:
#         experience_details = ', '.join([f"{exp.get('position', 'Unknown position')} at {exp.get('company', 'Unknown company')} ({exp.get('duration', 'Unknown duration')})" for exp in experience_list])
#     else:
#         experience_details = 'No experience provided'

#     # Extract project details
#     project_list = details.get('projects', [])
#     if project_list:
#         project_details = ', '.join([project.get('name', 'Unknown project') for project in project_list])
#     else:
#         project_details = 'No projects listed'

#     # Extract achievements details
#     achievement_list = details.get('achievements', [])
#     if achievement_list:
#         achievement_details = ', '.join([f"{achieve.get('position', 'Unknown position')} - {achieve.get('description', 'Unknown achievement')}" for achieve in achievement_list])
#     else:
#         achievement_details = 'No achievements listed'

#     email_prompt = f"""
#     You are {name}, educated at {education_details}. Your skills include {skills_details}.
    
#     You have experience as {experience_details}.
    
#     Some of your key projects include {project_details}.
    
#     Additionally, your achievements include {achievement_details}.
    
#     Write a cold email to a potential employer or client, showcasing your skills, education, projects, and achievements. Explain how your background makes you an ideal candidate for their needs.

#     Remember, you are {name}, ready to make a significant impact in your new role.
#     """
#     return email_prompt


In [83]:
# generate_cold_email(job)

In [84]:
# def generate_cold_email(details):
#     # Extract name
#     name = details.get('name', 'Candidate')

#     # Extract education details
#     education_list = details.get('education', [])
#     if education_list:
#         education_details = ', '.join([f"{edu.get('degree', 'Unknown degree')} from {edu.get('institution', 'Unknown institution')} ({edu.get('duration', 'Unknown duration')})" for edu in education_list])
#     else:
#         education_details = 'No education details provided'

#     # Extract skills details
#     skills_list = details.get('skills', [])
#     if skills_list:
#         skills_details = ', '.join([', '.join(skill.get('tools', [])) for skill in skills_list])
#     else:
#         skills_details = 'No skills listed'

#     # Extract experience details
#     experience_list = details.get('experience', [])
#     if experience_list:
#         experience_details = []
#         for exp in experience_list:
#             position = exp.get('position', 'Unknown position')
#             company = exp.get('company', 'Unknown company')
#             duration = exp.get('duration', 'Unknown duration')
#             achievements = exp.get('achievements', [])
#             achievements_details = ', '.join(achievements) if achievements else 'No achievements mentioned'
#             experience_details.append(f"{position} at {company} ({duration}): {achievements_details}")
#         experience_details = ', '.join(experience_details)
#     else:
#         experience_details = 'No experience provided'

#     # Extract project details
#     project_list = details.get('projects', [])
#     if project_list:
#         project_details = ', '.join([project.get('name', 'Unknown project') for project in project_list])
#     else:
#         project_details = 'No projects listed'

#     # Extract achievements details
#     achievement_list = details.get('achievements', [])
#     if achievement_list:
#         achievement_details = ', '.join([f"{achieve.get('position', 'Unknown position')} - {achieve.get('description', 'Unknown achievement')}" for achieve in achievement_list])
#     else:
#         achievement_details = 'No achievements listed'

#     email_prompt = f"""
#     You are {name}, educated at {education_details}. Your skills include {skills_details}.
    
#     You have experience as {experience_details}.
    
#     Some of your key projects include {project_details}.
    
#     Additionally, your achievements include {achievement_details}.
    
#     Write a cold email to a potential employer or client, showcasing your skills, education, experience (including responsibilities and achievements), projects, and achievements. Explain how your background makes you an ideal candidate for their needs.

#     Remember, you are {name}, ready to make a significant impact in your new role.
#     """
#     return email_prompt


In [86]:
# generate_cold_email(job)
# 

In [87]:
def generate_cold_email(details):
    # Extract name
    name = details.get('name', 'Candidate')

    # Extract education details
    education_list = details.get('education', [])
    if education_list:
        education_details = ', '.join([f"{edu.get('degree', 'Unknown degree')} from {edu.get('institution', 'Unknown institution')} ({edu.get('duration', 'Unknown duration')})" for edu in education_list])
    else:
        education_details = 'No education details provided'

    # Extract skills details
    skills_list = details.get('skills', [])
    if skills_list:
        skills_details = ', '.join([', '.join(skill.get('tools', [])) for skill in skills_list])
    else:
        skills_details = 'No skills listed'

    # Extract experience details
    experience_list = details.get('experience', [])
    if experience_list:
        experience_details = []
        for exp in experience_list:
            position = exp.get('position', 'Unknown position')
            company = exp.get('company', 'Unknown company')
            duration = exp.get('duration', 'Unknown duration')
            achievements = exp.get('achievements', [])
            achievements_details = ', '.join(achievements) if achievements else 'No achievements mentioned'
            experience_details.append(f"{position} at {company} ({duration}): {achievements_details}")
        experience_details = ', '.join(experience_details)
    else:
        experience_details = 'No experience provided'

    # Extract project details
    project_list = details.get('projects', [])
    if project_list:
        project_details = []
        for project in project_list:
            project_name = project.get('name', 'Unknown project')
            project_description = ', '.join(project.get('description', [])) if project.get('description') else 'No details provided'
            project_details.append(f"{project_name}: {project_description}")
        project_details = ', '.join(project_details)
    else:
        project_details = 'No projects listed'

    # Extract achievements details
    achievement_list = details.get('achievements', [])
    if achievement_list:
        achievement_details = ', '.join([f"{achieve.get('position', 'Unknown position')} - {achieve.get('description', 'Unknown achievement')}" for achieve in achievement_list])
    else:
        achievement_details = 'No achievements listed'

    email_prompt = f"""
    You are {name}, educated at {education_details}. Your skills include {skills_details}.
    
    You have experience as {experience_details}.
    
    Some of your key projects include {project_details}.
    
    Additionally, your achievements include {achievement_details}.
    
    Write a cold email to a potential employer or client, showcasing your skills, education, experience (including responsibilities and achievements), projects (with descriptions), and achievements. Explain how your background makes you an ideal candidate for their needs.

    Remember, you are {name}, ready to make a significant impact in your new role.
    """
    return email_prompt


In [89]:
generate_cold_email(job)

AttributeError: 'str' object has no attribute 'get'

In [88]:
resume_details = generate_cold_email(job)
type(resume_details)

AttributeError: 'str' object has no attribute 'get'

In [60]:
# job

In [57]:
# this is prompt template for writing an email

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        Introduce yourself from the below details
        {resume_details}
        End the email with Name and Designation. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

In [58]:
# again creating a chain of prompt_email and llm
# invoking the chain by passing the parameter of job_description and link_list

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "resume_details": resume_details})
print(res.content)

Subject: Expertise in AI, Machine Learning, and Deep Learning for Innovative Solutions

Dear Hiring Manager,

I am Mandar Bhalerao, a highly motivated and skilled professional with a strong educational background in Computer Science and Automation. I am excited to introduce myself as a potential candidate for a role that leverages my expertise in AI, Machine Learning, and Deep Learning.

With a Master of Technology degree in Computer Science and Automation from the Indian Institute of Science (Aug 2023 – Jul 2025) and a Bachelor of Engineering degree in Computer Engineering from P.E.S. Modern College of Engineering (Aug 2019 – Jul 2023), I possess a solid foundation in computer science and software development. My technical skills include proficiency in Python, C++, VS Code, Jupyter Notebook, Google Colab, Neural Networks, Machine Learning, Deep Learning, Gen AI, and Natural Language Processing (NLP).

As a Deep Learning Research Intern at NeuroPixel.AI (May 2024 – July 2024), I worked